In [1]:
# model used in my project
import math
import numpy as np
from sklearn import linear_model
import time



In [2]:
# this piece of code is copied directly from the blog. I use this decorator to test performance

import time
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print (  '{}  {} ms'.format(method.__name__, (te - ts) * 1000) )
        return result
    return timed

In [4]:
# Simple Least Squared Method Normal Equation
#Linear Regression 
class LinearRegression():
    def __init__(self,Y,X):
        #Y is an vecotor, X is a matrix, they are numpy object
        self.nobs=Y.shape[0]
        self.nvar=X.shape[1]+1 # intercept included
                
        if self.nobs<self.nvar:                
            raise ValueError("Degrees of freedom are less than 0, impossible to estimate all parameters")
        if np.linalg.matrix_rank(X)!=min(X.shape): 
            raise ValueError("Matrix X does not have full rank.Only non-singular matrix apply this method.")     
        if Y.shape[0]!=X.shape[0]: 
            raise ValueError("Dimensions of X and Y don't match. nrow of Y={}, nrow of X={}".format(Y.shape[0],X.shape[0]))       
        
        self.Y=Y
        self.X=np.insert(X,0,1,axis=1) # insert intercept into first column
        self.result={}     
            
    def __str__(self):
        return("Linear Regression Model: \nnumber of observations={}\nnumber of variables={} (intercept included)".format(self.Y.shape[0],self.X.shape[1]))
    def __repr__(self):
        return(self.__str__())
   
    @timeit
    def NormalEquation(self):
        ''' OLS normal equation method ： this formular require X to be full rank '''
        # run the complete regression include all the variables           
        X = self.X
        XtX_inv=np.linalg.inv(np.dot(X.T,X))
        self.result["OLS_Coefficients"]=np.dot(np.dot(XtX_inv,X.T),Y)
        self.result["OLS_Y_predict"]=np.dot(self.X,self.result["OLS_Coefficients"])
        self.result["OLS_Residuals"]=self.Y-self.result["OLS_Y_predict"]
        # AIC is just a statistical criteria to measure the performance of a model.
        self.result["OLS_AIC"]=self.nobs*np.power(self.result["OLS_Residuals"],2).sum()+self.nvar*2
        for keys,values in self.result.items():
            print(keys)
            print(values)       
    


In [5]:
X = np.array([[ 1, 4,90],
              [10, 5,8],
              [ 3, 9,10],
             [23,23,234]])
Y=np.array([1,2,3,10])
mymodel=LinearRegression(Y=Y,X=X)
mymodel.NormalEquation()

OLS_Coefficients
[-0.66389074  0.07519987  0.38124584  0.00070786]
OLS_Y_predict
[  1.   2.   3.  10.]
OLS_Residuals
[  1.07691633e-14  -8.88178420e-15   1.37667655e-14  -5.32907052e-15]
OLS_AIC
8.0
NormalEquation  3.009319305419922 ms


In [6]:
reg = linear_model.LinearRegression()
start_time = time.time()
reg.fit (X, Y)
print(reg.coef_)
print("Scikit-learn :  %s seconds " % (time.time() - start_time))

[ 0.07519987  0.38124584  0.00070786]
Scikit-learn :  0.0020008087158203125 seconds 


In [7]:
class ForwardSelection(LinearRegression):
    '''step-wise selection of variables, each time put the most correlated variable into the model   '''
    def __init__(self,Y,X):
        LinearRegression.__init__(self,Y=Y,X=X)
        self.active=[]
        self.inactive=[i for i in range(self.nvar)]
        self.coeset={0:np.array([0 for i in range(self.nvar)])}# initialize the first round coe
    
    def OLS(self,X=None):
        ''' OLS normal equation method ： this formular require X to be full rank '''  
        X= self.X if X is None else X
        XtX_inv=np.linalg.inv(np.dot(X.T,X))
        self.coe=np.dot(np.dot(XtX_inv,X.T),Y)
        return(self.coe)
    
    def GetResidual(self,coe=None):
        coe = self.coe if coe is None else coe
        if coe is None:
            coe = ForwardSelection.OLS(self)  
        return(self.Y-np.dot(self.X,coe)) 
               
    def stepwise(self):
        for j in range(self.nvar):
            last_residual=self.GetResidual(coe=self.coeset[j]) # residual from last run
            list_of_cov=[np.dot(self.X[:,i],last_residual) for i in self.inactive]  # see the all the covariance of redisual and inactive variables
            nextvar=self.inactive[list_of_cov.index(max(list_of_cov))] # choose the variable that is most correlated with the residuals
            
            # move this variable from inactive set to active set
            self.active.append(nextvar) 
            self.inactive.remove(nextvar)
            
            X=self.X[:,self.active]#picking all the variables in active set
            newcoe=ForwardSelection.OLS(self,X=X)# use this slice of X to run linear regression and get coeffecients
            # but the form [c,d,a,b] is not what we want, we would like to have form like this [0,a,0,b,c,d,0,0]
            finalcoe=[None for i in range(self.nvar)] 
            for i in range(len(finalcoe)):
                finalcoe[i]= newcoe[self.active.index(i)] if i in self.active else 0
            # store coeffecient result in a dict            
            self.coeset[j+1]=finalcoe
        return(self.coeset)              
        


In [8]:
mymodel=ForwardSelection(Y=Y,X=X)
mymodel.stepwise()

{0: array([0, 0, 0, 0]),
 1: [0, 0, 0, 0.039289114566804187],
 2: [0, 0.28143477550791052, 0, 0.014361013695887676],
 3: [0, 0.087098869903475748, 0.31256846241576219, 0.0024502078212224887],
 4: [-0.66389073950699551,
  0.075199866755496447,
  0.38124583610926144,
  0.00070786142571611574]}

In [9]:
import pandas as pd
import os
# I would like to test my stepwise feature forward selection model for data from real world.
# this data set is download from kaggle.com https://www.kaggle.com/c/liberty-mutual-fire-peril
# 1% was drawn from the original data for convenience.
ep=os.path.expanduser('~/python_project/train_sample.csv')
os.path.normpath(ep)
train = pd.read_csv(os.path.normpath(ep))

In [10]:
# Y is the ratio of loss due to the fire. X is weather conditions
X=train.filter(regex="weatherVar[0-9]{1}$") # subtract the weather data set
X.head()

,weatherVar1,weatherVar2,weatherVar3,weatherVar4,weatherVar5,weatherVar6,weatherVar7,weatherVar8,weatherVar9
0,7.951660,0.0,0.882133,1.635643,0.420159,0.923142,0.663614,9.497982,28.999883
1,0.446103,0.0,1.147721,0.009012,1.157393,0.015541,1.187234,0.609595,0.000000
2,0.204195,0.0,0.919459,0.574934,1.246018,1.247934,0.949284,0.000000,0.000000
3,0.361528,0.0,0.983296,0.275168,0.200429,0.146865,0.632942,0.000000,0.000000
4,0.205732,0.0,1.128833,0.390005,1.222026,0.003105,1.200807,0.000000,0.000000


In [11]:
X=train.filter(regex="weatherVar[0-9]{1}$").values # treat as numpy arrays
Y=train['target'].values
sampledata=np.insert(X,0,Y,axis=1)
sampledata=sampledata[~np.isnan(sampledata).any(1)] # drop the NAN values
Y=sampledata[:,0] # first column is Y
X=sampledata[:,1:] # columns other than first are X
X.shape # number of the observation , number of the variable

(4486, 9)

In [12]:
# use packages module to dig in first
reg = linear_model.LinearRegression(fit_intercept=True)
start_time = time.time()
reg.fit (X, Y)
print(reg.coef_)
print("--- %s seconds ---" % (time.time() - start_time))

[ -1.33345153e-03  -7.86503200e-06  -6.10740128e-03  -1.27538322e-03
   7.17345310e-03  -1.53538037e-04  -1.23258110e-02  -1.06642283e-04
  -2.96953045e-04]
--- 0.006014823913574219 seconds ---


In [14]:
# see my model is function well, with more details 
mymodel=LinearRegression(Y=Y,X=X)
mymodel.NormalEquation()

OLS_Coefficients
[  2.27131565e-02  -1.33345153e-03  -7.86503200e-06  -6.10740128e-03
  -1.27538322e-03   7.17345310e-03  -1.53538037e-04  -1.23258110e-02
  -1.06642283e-04  -2.96953045e-04]
OLS_Y_predict
[-0.01029543  0.00869871  0.01313807 ...,  0.00405624  0.00387245
  0.00913794]
OLS_Residuals
[ 0.01029543 -0.00869871 -0.01313807 ..., -0.00405624 -0.00387245
 -0.00913794]
OLS_AIC
1013936.36604
NormalEquation  5.011796951293945 ms


In [15]:
#forward selection model
mymodel=ForwardSelection(Y=sampledata[:,0],X=sampledata[:,1:])
for key,val in mymodel.stepwise().items():
    print(key)
    print(val)

0
[0 0 0 0 0 0 0 0 0 0]
1
[0, 0, 0.00031676876579415469, 0, 0, 0, 0, 0, 0, 0]
2
[0, 0, 2.5572863052667302e-05, 0, 0, 0.0079425454161444423, 0, 0, 0, 0]
3
[0.0014825179451017456, 0, 2.825848175163711e-05, 0, 0, 0.0066054168686668635, 0, 0, 0, 0]
4
[0.0079135446887661897, 0, 1.6158246015962224e-05, -0.0060191772267407977, 0, 0.0062656120373040776, 0, 0, 0, 0]
5
[0.0072288895147807619, 0, 1.3782623755335046e-05, -0.0023203151032182186, 0, 0.0068567659785199906, 0, -0.0036167084979231694, 0, 0]
6
[0.0082720424423057895, 0, 1.3406225899960691e-05, -0.0023667674564541337, 0, 0.0073637838623781668, -0.00033848806746898224, -0.0047871844441813236, 0, 0]
7
[0.016933339297769075, -0.0019335289611352782, 2.9661336499196379e-06, -0.0023835306722729786, 0, 0.0076166353231447454, -0.00041622791290102346, -0.011660178773153215, 0, 0]
8
[0.015978984652964781, -0.0018504417616847233, 2.5502259389063553e-06, 0.0015500404695541099, 0, 0.0077276692706368267, -0.00042105029142126474, -0.014749239973317599,